In [1]:
#获取用户对达人的评分
def getUserRate(self):
    userRateDict = dict()
    fr = open(self.user_rate_file, "r", encoding='utf-8')
    for line in fr.readlines():
        if not line.startswith('userID'):
            userID, artistID, weight = line.split('\t')
            userRateDict.setdefault(int(userID), {})
            #对听歌次数缩放，避免计算结果太大
            userRateDict[int(userID)][int(artistID)] = float(weight) / 10000           
    return userRateDict


In [4]:
#获得每个用户打标对标签和每个标签被打标的次数
def getUserTagNum(self):
    userTagDict = dict()
    tagUserDict = dict()
    for line in open(self.user_tag_file, "r", encoding='utf-8'):
        if not line.startswith('userID'):
            userID, artistID, weight = line.strip().split('\t')[:3]
            #每个标签被打标的次数
            if int(tagID) in tagUserDict.keys():
                tagUserDict[int(tagID)] += 1
            else:
                tagUserDict[int(tagID)] = 1
            #每个用户对每个标签的打标次数
            userTagDict.setdefault(int(userID), {})
            if int(tagID) in userTagDict[int(userID)].keys():
                userTagDict[int(userID)][int(tagID)] += 1
            else:
                userTagDict[int(userID)][int(tagID)] = 1
    return userTagDict, tagUserDict
                

In [1]:
# 获取艺术家对应的标签基因,这里的相关度全部为1
def getArtistsTags(self):
    artistsTagsDict = dict()
    for line in open(self.user_tag_file, 'r', encoding='utf-8'):
        if not line.startswith('userID'):
            artistID, tagID = line.strip('\t')[1:3]
            artistsTagsDict.setdefault(int(artistID), {})
            # 如果艺术家有对应的标签则记录，相关度为1，否则不为1
            artistsTagsDict[int(artistID)][int(tagID)] = 1
    return artistsTagsDict
            

In [2]:
# 获取用户对标签的最终兴趣度
def getUserTagPre(self):
    userTagPre = dict()
    userTagCount = dict()
    # Num 为用户打标总条数
    Num = len(open(self.user_tag_file, "r", encoding="utf-8").readlines())
    for line in open(self.user_tag_file, "r", encoding="utf-8").readlines():
        if not line.startswith('userID'):
            userID, artistID, tagID = line.strip('\t')[:3]
            userTagPre.setdefault(int(userID), {})
            userTagCount.setdefault(int(userID), {})
            #rate_ui为用户对艺术家对评分
            rate_ui = (
                self.userRateDict[int(userID)][int(artistID)]
                if int(artistID) in self.userRateDict[int(userID)].keys()
                else 0
            )
            #artistsTagsDict艺术家对标签的相关度，userTagPre用户对标签的主观喜好
            if int(tagID) not in userTagPre[int(userID)].keys():
                userTagPre[int(userID)][int(tagID)] = (
                    rate_ui * self.artistsTagsDict[int(artistID)][int(tagID)]
                    
                )
                userTagCount[int(userID)][int(tagID)] = 1
            else:
                userTagPre[int(userID)][int(tagID)] += (
                        rate_ui * self.artistsTagsDict[int(artistID)][int(tagID)]
                    )
                userTagCount[int(userID)][int(tagID)] += 1
    
    for userID in userTagPre.keys():
        for tagID in userTagPre[userID].keys():
            #tfidf值为用户对标签的依赖度
            #计算tf值
            tf_ut = self.userTagDict[int(userID)][int(tagID)] / sum(self.userTagDict[int(userID)].values())
            #计算idf值
            idf_ut = math.log(Num * 1.0 / (self.tagUserDict[int(tagID)] + 1))
            #用户对标签的兴趣度为主观喜好度*依赖度
            userTagPre[userID][tagID] = (
                userTagPre[userID][tagID] / userTagCount[userID][tagID] * tf_ut * idf_ut
            )
        return userTagPre
            
